# Reanding Exif data from ./images

In [70]:
import os
from PIL import Image

dir = "images"
themes = os.listdir(dir)
data = []

for theme in themes: 
    if theme in {".DS_Store", "ExifData.json","ExifData2.json","ExifDatatest.json"}:
        pass   
    else:
        print(theme)
        dir_test = os.path.join(dir, theme)
        for image in os.listdir(dir_test):
            #print(image)
            if image.endswith(".jpg") :
                #img_path =  dir + theme + '/'+ image
                img_path = os.path.join(dir, theme, image)
                # print(img_path)
                with Image.open(img_path) as img:
                    # print(img)
                    width, height = img.size
                    format = img.format
                    try:
                        orientation = img._getexif()[274]
                    except:
                        orientation = "no exif"
                    #print(f'{width=}x{height=} {format=} {orientation=}')
                    data.append({"theme": theme, "image": image, "width": width, "height": height, "format": format, "orientation": orientation})
                    
print(data[0:5])

Exoplanet
Pokemon
Moto
Voitures
[{'theme': 'Exoplanet', 'image': '55_Cancri_b.jpg', 'width': 1024, 'height': 768, 'format': 'PNG', 'orientation': 'no exif'}, {'theme': 'Exoplanet', 'image': 'Gliese_876_e.jpg', 'width': 1587, 'height': 919, 'format': 'PNG', 'orientation': 'no exif'}, {'theme': 'Exoplanet', 'image': 'XO-3b.jpg', 'width': 800, 'height': 500, 'format': 'JPEG', 'orientation': 'no exif'}, {'theme': 'Exoplanet', 'image': 'OGLE-2005-BLG-169Lb.jpg', 'width': 1271, 'height': 746, 'format': 'JPEG', 'orientation': 'no exif'}, {'theme': 'Exoplanet', 'image': 'Kepler-242b.jpg', 'width': 1980, 'height': 1200, 'format': 'JPEG', 'orientation': 'no exif'}]


## Get proiminant colors from images

In [71]:
from PIL import Image
import warnings
import numpy as np
import math
import matplotlib.pyplot as plot
from sklearn.cluster import MiniBatchKMeans

def getKmeans(img):
    # init
    img = Image.open(img) # open image
    imgfile = img.resize((int(img.size[0]/5), int(img.size[1]/5))) # resize
    numarray = np.array(imgfile.getdata(), np.uint8) # convert image to numpy array
    cluster_count = 5 # number of clusters
    clusters = MiniBatchKMeans(n_clusters=cluster_count, n_init="auto") # init kmeans
    clusters.fit(numarray) # fit kmeans
    npbins = np.arange(0, cluster_count + 1) # init bins
    histogram = np.histogram(clusters.labels_, bins=npbins) # get histogram
    #labels = numpy.unique(clusters.labels_) # get labels

    # getting colors
    color = []
    for i in range(cluster_count):
        color.append(
            "#%02x%02x%02x" % (math.ceil(clusters.cluster_centers_[i][0]),math.ceil(clusters.cluster_centers_[i][1]),math.ceil(clusters.cluster_centers_[i][2]),)
        )
    #barlist = plot.bar(labels, sorted(histogram[0], reverse=True), color=color)
    #plot.show()
    return (color)


getKmeans("./images/Exoplanet/2M1207_b.jpg")

['#1c1e20', '#a49486', '#4d4136', '#0a0b0d', '#826b58']

## Transformation de hexa a RGB

In [72]:
import numpy as np

list_of_colors = [
                (204,0,0), #red
                (204,102,0),# brown
                (204,204,0),# yellow
                (102,204,0),# lightgreen
                (0,204,0),# green
                (0,204,102),# bluegreen
                (0,204,204),# lightblue
                (0,102,204), # blue
                (0,0,204), # darkblue
                (102,0,204), # purple
                (204,0,204), # pink
                (204,0,102), # darkpink
                (255,255,255), # white
                (192,192,192), # lightgrey
                (128,128,128), # grey
                (64,64,64), # darkgrey
                (0,0,0) # black
]

color_to_name = ["red", "brown", "yellow", "lightgreen","green","bluegreen", "lightblue","blue","darkblue", "purple", "pink","darkpink", "white","lightgrey", "grey","darkgrey", "black"]

def color_hex_to_rgb(color):
    return list(int(color.lstrip('#')[i:i+2], 16) for i in (0, 2, 4))

def rgb_to_name(color_to_name,list_of_colors ,rgb):
    return color_to_name[list_of_colors.index(tuple(rgb))]


def closest_color(colors,color_hexa):
    color = color_hex_to_rgb(color_hexa)
    colors = np.array(colors)
    color = np.array(color)
    distances = np.sqrt(np.sum((colors-color)**2,axis=1))
    index_of_smallest = np.where(distances==np.amin(distances))
    smallest_distance = colors[index_of_smallest]
    return smallest_distance 

color = "#ff00b0"
rgb = closest_color(list_of_colors,color)
print(rgb[0])
name = rgb_to_name(color_to_name,list_of_colors,rgb[0])
print(name)

[204   0 204]
pink


## Getting histogram from images

In [73]:
from PIL import Image
import matplotlib.pyplot as plot
import numpy as np

def getDiagram(img):
    imgfile = Image.open(img)
    histogram = imgfile.histogram()
    red = histogram[0:255]
    green = histogram[256:511]
    blue = histogram[512:767]
    # x = range(255)
    # y = []
    # for i in x:
    #     y.append((red[i], green[i], blue[i]))
    # figure,axes = plot.subplots()
    # axes.set_prop_cycle("color", ["red", "green", "blue"])
    # plot.plot(x, y)
    # plot.show()
    mapping = {"red": np.average(range(0,255),weights=red), "green": np.average(range(0,255),weights=green), "blue": np.average(range(0,255),weights=blue)}
    return mapping

getDiagram("./images/Exoplanet/2M1207_b.jpg")

{'red': 44.77778330018979,
 'green': 41.671026930561624,
 'blue': 39.357752621028695}

## Adding diagram and Kmeans to exif data

In [75]:
from tqdm import tqdm

for i in tqdm(range(len(data))):
    dir = os.path.join("images", data[i]["theme"], data[i]["image"])
    colors = getKmeans(dir)
    for n, c in enumerate(colors):
        rgb =(closest_color(list_of_colors,c))[0]
        data[i][f"color_{n}"] = rgb_to_name(color_to_name, list_of_colors, rgb)
    RGB = getDiagram(dir)
    for key, value in RGB.items():
        data[i][key] = value

100%|██████████| 355/355 [02:42<00:00,  2.19it/s]


## Writing exif data into json file : ./images/ExifData.json

In [ ]:
import json
from numpyencoder import NumpyEncoder

with open('./images/ExifDatatest.json', 'w+') as f:
    json.dump(data, f, indent=4, cls=NumpyEncoder)